In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1001_z"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/synthetic2/' + 'processed4.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
train_data.shape, validation_data.shape, test_data.shape, train_gt.shape, validation_gt.shape, test_gt.shape, index_of_changes.shape

((900, 8192, 12),
 (100, 8192, 12),
 (1000, 8192, 12),
 (900, 8192, 1),
 (100, 8192, 1),
 (1000, 8192, 1),
 (1000, 4))

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]
bias_shape = [32, 64, 64, 64, 64]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

#variables_weights = np.ones((n_classes,))

In [7]:
print pos_weight

2.46045164873


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../../summary_synthetic/train_gradual_test_abrupt/' + exp_name 
checkpoint_path = '../../model_synthetic/train_gradual_test_abrupt/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight_values= pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
AUC = 0.255 vs. 0.280
F1 = 0.440 vs. 0.429
Precision = 0.285 vs. 0.273
Recall = 0.967 vs. 0.999
Validation: 0
AUC = 0.271 vs. 0.288
F1 = 0.476 vs. 0.445
Precision = 0.320 vs. 0.286
Recall = 0.932 vs. 1.000
Minibatch Loss= 0.986096
Training: 100
AUC = 0.520 vs. 0.278
F1 = 0.499 vs. 0.439
Precision = 0.386 vs. 0.282
Recall = 0.704 vs. 0.989
Validation: 100
AUC = 0.589 vs. 0.291
F1 = 0.531 vs. 0.445
Precision = 0.391 vs. 0.286
Recall = 0.829 vs. 0.999
Minibatch Loss= 0.931215
Training: 200
AUC = 0.754 vs. 0.278
F1 = 0.670 vs. 0.439
Precision = 0.729 vs. 0.282
Recall = 0.619 vs. 0.996
Validation: 200
AUC = 0.737 vs. 0.288
F1 = 0.636 vs. 0.445
Precision = 0.639 vs. 0.286
Recall = 0.633 vs. 0.997
Minibatch Loss= 0.789035
Training: 300
AUC = 0.793 vs. 0.291
F1 = 0.690 vs. 0.451
Precision = 0.720 vs. 0.291
Recall = 0.662 vs. 1.000
Validation: 300
AUC = 0.769 vs. 0.290
F1 = 0.672 vs. 0.445
Precision = 0.729 vs. 0.286
Recall = 0.624 vs. 1.000
Minibatch Loss= 0.731866
Training: 400
AU

In [ ]:
trainer.train(max_iter = 10000,
                train_eval_step = 1000, 
                validation_eval_step = 1000,
                display_step = 1000)

In [15]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model_synthetic/train_gradual_test_abrupt/CNN1001_z/CNN1001_z-10000'

In [16]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../models/model" + exp_name + "/model")

In [19]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.756 vs. 0.074
F1 = 0.759 vs. 0.137
Precision = 0.756 vs. 0.073
Recall = 0.761 vs. 0.999


In [20]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [21]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
out = np.squeeze(out, axis = 2)

In [22]:
out.shape

(1000, 256)

In [23]:
savemat('../../poc_synthetic/train_gradual_test_abrupt/poc'+exp_name+'.mat', {'poc':out})